In [ ]:
!pip install torch 
!pip install torchvision
!pip install transformers
!pip install evaluate
!pip install albumentations
None

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 7.6 MB/s 
     |████████████████████████████████| 182 kB 73.7 MB/s 
     |████████████████████████████████| 7.6 MB 47.0 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 72 kB 1.4 MB/s 
     |████████████████████████████████| 115 kB 14.8 MB/s 
     |████████████████████████████████| 451 kB 59.4 MB/s 
     |████████████████████████████████| 212 kB 64.8 MB/s 
     |████████████████████████████████| 127 kB 68.0 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [166]:
import torch 
import cv2
from transformers import AutoFeatureExtractor, ResNetForImageClassification, Trainer, TrainingArguments
import evaluate
import matplotlib.pyplot as plt
from torch import nn
from torch import optim
import torch.nn.functional as F
from pandas.core.common import flatten

from torchvision import datasets, transforms, models
from torch.utils.data import Dataset, DataLoader
from torchvision.datasets import ImageFolder

import albumentations as A
from albumentations.pytorch import ToTensorV2
import glob
from tqdm import tqdm

import os
import numpy as np

import warnings
import random
from torchsummary import summary
warnings.filterwarnings('ignore')

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [178]:
train_path = '/content/drive/MyDrive/GeorgeDataSet/train'
test_path = '/content/drive/MyDrive/GeorgeDataSet/test'

In [179]:
train_path = '/content/drive/MyDrive/GeorgeDataSet/train'
test_path = '/content/drive/MyDrive/GeorgeDataSet/test'

In [180]:
# for generating paths
import os
classes = ['nofire', 'fire']

if os.path.exists('trainpaths.txt'):
  os.remove('trainpaths.txt')

root_dir = train_path
for folder in os.listdir(root_dir):
  for filename in os.listdir(os.path.join(root_dir, folder)):
    path = os.path.join(root_dir, folder, filename)
    label = classes.index(folder)
    with open('trainpaths.txt','a') as f:
      f.writelines(str(label) + ',' + path +'\n')

if os.path.exists('testpaths.txt'):
  os.remove('testpaths.txt')

root_dir = test_path
for folder in os.listdir(root_dir):
  for filename in os.listdir(os.path.join(root_dir, folder)):
    path = os.path.join(root_dir, folder, filename)
    label = classes.index(folder)
    with open('testpaths.txt','a') as f:
      f.writelines(str(label) + ',' + path +'\n')


In [ ]:
train_dataset = ImageFolder('/content/drive/MyDrive/GeorgeDataSet/train',transform=None)
test_dataset = ImageFolder('/content/drive/MyDrive/GeorgeDataSet/test', transform=None)

In [181]:
class CustomDataset(Dataset):
  def __init__(self, path_file, transform=None):
    self.transform = transform
    with open(path_file, 'r') as f:
      self.paths = f.readlines()

  def __len__(self):
    return len(self.paths)

  def __getitem__(self, idx):
    label, img_path = self.paths[idx].strip().split(',')
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    if self.transform:
      img = self.transform(img)
    return img, int(label)

In [185]:
trainset_configs = dict(
    path_file = 'trainpaths.txt',
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(30),
        transforms.RandomVerticalFlip(),
        transforms.Resize((224,224)),
        transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),

    ])
)

testset_configs = dict(
    path_file = 'testpaths.txt',
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Resize((224,224)),
        transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
        
    ])
)

In [186]:
train_dataset = CustomDataset(
    path_file = trainset_configs['path_file'],
    transform = trainset_configs['transform']
      )

test_dataset = CustomDataset(
    path_file = testset_configs['path_file'],
    transform = testset_configs['transform']
)

In [187]:
train_dataloader = DataLoader(
    dataset=train_dataset,
    batch_size=64,
    shuffle=True,
    num_workers=2,
    pin_memory = torch.cuda.is_available()
)

test_dataloader = DataLoader(
    dataset=test_dataset,
    batch_size=64,
    shuffle=True,
    num_workers=2,
    pin_memory = torch.cuda.is_available()
)

In [ ]:
iterator = iter(train_dataloader)
images, labels = next(iterator)

In [188]:
from torchvision import models
import torch.nn as nn

model = models.resnet18(pretrained=True)
model.fc = nn.Sequential(
    nn.Linear(512,2)

)
model = model.to(device)

In [151]:
from torchvision import models
import torch.nn as nn

model = models.resnet50(pretrained=True)
# for child in model.children():
#   for param in child.parameters():
#     param.requires_grad = False
model.fc = nn.Sequential(
    nn.Linear(2048,1024),
    nn.Linear(1024,512),
    nn.Linear(512,256),
    
)
model = model.to(device)



In [ ]:
device

'cuda'

In [189]:
import torch.optim as optim
from torch.optim import lr_scheduler
loss_function = nn.CrossEntropyLoss()

optimiser = optim.Adam([
    {'params': model.conv1.parameters()},
    {'params': model.layer1.parameters()},
    {'params': model.layer2.parameters()},
    {'params': model.layer3.parameters()},
    {'params': model.layer4.parameters()},
    {'params': model.fc.parameters(), 'lr':0.01}
    ],
  lr=0.0001, 
  weight_decay=0.0005

  # weight_decay=0.0005
)
scheduler = lr_scheduler.StepLR(optimiser, step_size=10, gamma=0.1)

In [ ]:
!pip install torchmetrics

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 529 kB 7.6 MB/s 


In [190]:
#train.py file
from sklearn.metrics import f1_score 
import time
import torch
import copy
from torchmetrics import F1Score
from torch.utils.tensorboard import SummaryWriter

device = 'cuda' if torch.cuda.is_available() else 'cpu'

def train_model(model, dataloaders, criterion, optimiser, scheduler, num_epochs):
  best_acc = 0
  since = time.time()
  writer = SummaryWriter()
  
  for epoch in range(num_epochs):
    print('Epoch{}'.format(epoch))
    for phase in ['train', 'test']:
      epoch_loss = 0
      epoch_acc = 0
      f1_score = 0
      if phase == 'train':
        model.train()
      else:
        model.eval()

      num_images = 0
      batch = 0
      for inputs, gt_labels in dataloaders[phase]:
        batch+=1
        inputs = inputs.to(device)
        gt_labels = gt_labels.to(device)

        optimiser.zero_grad()

        with torch.set_grad_enabled(phase=='train'):
          outputs = model(inputs)
        _,preds = torch.max(outputs, dim=1)
        loss = criterion(outputs, gt_labels)

        if phase == 'train':
          loss.backward()
          optimiser.step()

        num_images += inputs.shape[0]
        epoch_loss += loss.item()*inputs.shape[0]
        epoch_acc += torch.sum(preds==gt_labels).item() 

        

      if phase == 'train':
        scheduler.step()
      epoch_loss/= num_images
      epoch_acc /= num_images
   


      print('{}. Loss: {:.4f}, Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))
      
      if phase =='test' and epoch_acc > best_acc:
        best_acc = epoch_acc
        best_weights = copy.deepcopy(model.state_dict())
        torch.save(model.state_dict(), 'checkpoint.pt') 

      if phase ==  'train':
        writer.add_scalar('TrainLoss', epoch_loss, epoch)
        writer.add_scalar ('TrainAcc', epoch_acc, epoch)

      if phase =='test':
        writer.add_scalar ('ValLoss', epoch_acc, epoch)
        writer.add_scalar ('ValAcc', epoch_acc, epoch)

  time_elapsed = time.time() - since
  print('Training complete in {:.0f}m, {:.0f}s'.format(
      time_elapsed//60, time_elapsed%60
  ))

  model.load_state_dict(best_weights)
  return model
  

In [ ]:
class EarlyStopper:
    def __init__(self, patience=1, min_delta=0):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.min_validation_loss = np.inf

    def early_stop(self, validation_loss):
        if validation_loss < self.min_validation_loss:
            self.min_validation_loss = validation_loss
            self.counter = 0
        elif validation_loss > (self.min_validation_loss + self.min_delta):
            self.counter += 1
            if self.counter >= self.patience:
                return True
        return False

In [191]:
dataloaders = dict(
    train = train_dataloader,
    test = test_dataloader
)

trained_model = train_model(model,
                            dataloaders,
                            loss_function,
                            optimiser,
                            scheduler,
                            num_epochs=10
                            )

Epoch0
train. Loss: 0.3846, Acc: 0.8922
test. Loss: 0.0291, Acc: 0.9904
Epoch1
train. Loss: 0.0550, Acc: 0.9819
test. Loss: 0.0298, Acc: 0.9828
Epoch2
train. Loss: 0.0440, Acc: 0.9871
test. Loss: 0.0594, Acc: 0.9771
Epoch3
train. Loss: 0.0225, Acc: 0.9900
test. Loss: 0.0529, Acc: 0.9809
Epoch4
train. Loss: 0.0140, Acc: 0.9962
test. Loss: 0.0335, Acc: 0.9866
Epoch5


KeyboardInterrupt: ignored

In [136]:
torch.save(trained_model, 'resnet50_finetuned.pt')

In [ ]:
torch.save(trained_model, 'resnet50.pt')

In [ ]:
model = torch.load('resnet_finetuned.pt')
model.eval()

In [192]:
import torch 
import torchvision
from torchvision import models, transforms
from PIL import Image
import pandas as pd


import os
import numpy as np

import glob
import random
import time

device = 'cuda' if torch.cuda.is_available() else 'cpu'

PATH_MODEL = r'resnet50_finetuned.pt'
PATH_DATA = r'/content/drive/MyDrive/GeorgeDataSet/test_imgs'
PATH_OUTPUT = r'results.csv'

if __name__ =='__main__':

    
    model = torch.load(PATH_MODEL)
    model.eval()
    classes = ['nofire', 'fire']
    classes_to_idx = {
        'nofire' : 0,
        'fire' : 1
    }

    transformations=transforms.Compose([
                 transforms.Resize(size=224),
                #  transforms.CenterCrop(size=224),
                 transforms.ToTensor(),
                 transforms.Normalize([0.485, 0.456, 0.406],
                                      [0.229, 0.224, 0.225])
    ])
    
    img_paths = []
    images_paths = []
    
    for path in  os.listdir(PATH_DATA):
        img_paths.append(os.path.join(PATH_DATA, path)) #for loading images
        images_paths.append(path)                       #for result.csv

    
    result_labels = []                                  #for result.csv
    with torch.no_grad():
        for num, img_path in enumerate(img_paths):
            img = Image.open(img_path).convert('RGB')
            inputs = transformations(img).unsqueeze(0).to(device)
            outputs = model(inputs)
            print(torch.softmax(outputs, dim=1))
            _, preds = torch.max(outputs, dim=1)
            result_labels.append(classes_to_idx[classes[preds]])

    result = pd.DataFrame()
    result['img'] = images_paths
    result['target'] = result_labels
    result.to_csv(PATH_OUTPUT, index=False,header=False)
    print('Success!\nResults saved at {}'.format(PATH_OUTPUT))
    

tensor([[0.0365, 0.9635]], device='cuda:0')
tensor([[0.0260, 0.9740]], device='cuda:0')
tensor([[9.9984e-01, 1.6032e-04]], device='cuda:0')
tensor([[1.0000e+00, 5.1443e-07]], device='cuda:0')
tensor([[1.8728e-05, 9.9998e-01]], device='cuda:0')
tensor([[1.0000e+00, 5.0394e-08]], device='cuda:0')
tensor([[9.5714e-06, 9.9999e-01]], device='cuda:0')
tensor([[2.7458e-07, 1.0000e+00]], device='cuda:0')
tensor([[1.4018e-05, 9.9999e-01]], device='cuda:0')
tensor([[9.1356e-07, 1.0000e+00]], device='cuda:0')
tensor([[1.6721e-06, 1.0000e+00]], device='cuda:0')
tensor([[0.9990, 0.0010]], device='cuda:0')
tensor([[1.7221e-07, 1.0000e+00]], device='cuda:0')
tensor([[4.8246e-08, 1.0000e+00]], device='cuda:0')
tensor([[5.8980e-04, 9.9941e-01]], device='cuda:0')
tensor([[0.0108, 0.9892]], device='cuda:0')
tensor([[2.4730e-04, 9.9975e-01]], device='cuda:0')
tensor([[3.1590e-04, 9.9968e-01]], device='cuda:0')
tensor([[0.0191, 0.9809]], device='cuda:0')
tensor([[1.0000e+00, 1.4655e-10]], device='cuda:0')


**Data augmenting and File handling**

---



In [193]:
import os, glob, csv, random
import shutil
fire = []
nofire = []
with open('results.csv', newline='') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=' ', quotechar='|')
    for row in spamreader:
        img, label = row[0].split(",")
        if int(label) == 1:
          fire.append(img)
        else:
          nofire.append(img)

print(len(fire) + len(nofire))
# dst1 = "1"
# dst0 = "0"
# for file in os.listdir('train_imgs'):
#     if file in fire:
#         shutil.move(os.path.join('train_imgs',file),os.path.join('1',file))
#     else:
#         shutil.move(os.path.join('train_imgs',file),os.path.join('0',file))

# source = 'train/1'
# dest = 'val/1'
# files = os.listdir(source)
# no_of_files = len(files) // 5

# for file_name in random.sample(files, no_of_files):
#     shutil.move(os.path.join(source, file_name), dest)
# to_move = ['1640.jpg', '2111.jpg', '2931.jpg', '3098.jpg', '3493.jpg', '6710.jpg','7639.jpg', '8472.jpg', '9331.jpg']
# for file in os.listdir('/content/drive/MyDrive/GeorgeDataSet/test_imgs'):
#   if file in fire:
#     shutil.copy(os.path.join('/content/drive/MyDrive/GeorgeDataSet/test_imgs', file), os.path.join('/content/drive/MyDrive/GeorgeDataSet/pred4/fire', file))
#   else:
#     shutil.copy(os.path.join('/content/drive/MyDrive/GeorgeDataSet/test_imgs', file), os.path.join('/content/drive/MyDrive/GeorgeDataSet/pred4/nofire', file))

400


In [ ]:
from google.colab.patches import cv2_imshow
for file in os.listdir('/content/drive/MyDrive/GeorgeDataSet/alltrain/nofire'):
  shutil.copy(os.path.join('/content/drive/MyDrive/GeorgeDataSet/alltrain/nofire', file), os.path.join('/content/drive/MyDrive/GeorgeDataSet/alltrain_aug/nofire', file))
  img = cv2.imread(os.path.join('/content/drive/MyDrive/GeorgeDataSet/alltrain/nofire',file))
  img = cv2.flip(img,1)
  newfile = file.split(".")[0]+"aug.jpg"
  cv2.imwrite(os.path.join('/content/drive/MyDrive/GeorgeDataSet/alltrain_aug/nofire',newfile),img)



In [121]:
def rotation(img, angle):
    angle = int(random.uniform(-angle, angle))
    h, w = img.shape[:2]
    M = cv2.getRotationMatrix2D((int(w/2), int(h/2)), angle, 1)
    img = cv2.warpAffine(img, M, (w, h))
    return img
def channel_shift(img, value):
    value = int(random.uniform(-value, value))
    img = img + value
    img[:,:,:][img[:,:,:]>255]  = 255
    img[:,:,:][img[:,:,:]<0]  = 0
    img = img.astype(np.uint8)
    return img

In [122]:
from google.colab.patches import cv2_imshow
for file in os.listdir('/content/drive/MyDrive/GeorgeDataSet/alltrain/nofire'):
  img = cv2.imread(os.path.join('/content/drive/MyDrive/GeorgeDataSet/alltrain/nofire',file))
  #img = cv2.flip(img,1)
  # img = rotation(img, 30) 
  img = channel_shift(img, 60)
  newfile = file.split(".")[0]+"chsh.jpg"
  cv2.imwrite(os.path.join('/content/drive/MyDrive/GeorgeDataSet/alltrain_aug/nofire',newfile),img)



In [123]:
len(os.listdir('/content/drive/MyDrive/GeorgeDataSet/alltrain_aug/nofire'))

4266

In [120]:
len(os.listdir('/content/drive/MyDrive/GeorgeDataSet/alltrain_aug/fire'))

4656